# ML-ZC Homework 5

# Question 1

In [3]:
import subprocess


def get_pipenv_version():
    try:
        # Run pipenv --version command
        result = subprocess.run(['pipenv', '--version'], capture_output=True, text=True)
        
        # Check if the command was successful
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            return f"Error: {result.stderr}"
            
    except FileNotFoundError:
        return "Pipenv is not installed. Please install it first using 'pip install pipenv'"

if __name__ == "__main__":
    version = get_pipenv_version()
    print(version)

pipenv, version 2024.2.0


# Question 2

In [5]:
import subprocess
import json
import os

def install_sklearn_and_get_hash():
    try:
        # Install scikit-learn with specific version using pipenv
        print("Installing scikit-learn 1.5.2...")
        subprocess.run(['pipenv', 'install', 'scikit-learn==1.5.2'], check=True)
        
        # Read Pipfile.lock
        if os.path.exists('Pipfile.lock'):
            with open('Pipfile.lock', 'r') as f:
                lockfile = json.load(f)
                
            # Get the first hash for scikit-learn
            sklearn_info = lockfile.get('default', {}).get('scikit-learn', {})
            hashes = sklearn_info.get('hashes', [])
            
            if hashes:
                return f"First hash for scikit-learn: {hashes[0]}"
            else:
                return "No hashes found for scikit-learn"
        else:
            return "Pipfile.lock not found"
            
    except subprocess.CalledProcessError as e:
        return f"Error installing scikit-learn: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"

if __name__ == "__main__":
    result = install_sklearn_and_get_hash()
    print(result)

Installing scikit-learn 1.5.2...


We recommend setting this in ~/.profile (or equivalent) for proper expected behavior.
Creating a virtualenv for this project
Pipfile: /Users/sebastianreyes/Documents/3 
Carrera/Proyectos/Portafolio/machinelearning_zoomcamp/Pipfile
Using default python from /usr/local/bin/python33.10.7 to create virtualenv...
created virtual environment CPython3.10.7.final.0-64 in 447ms
  creator 
CPython3Posix(dest=/Users/sebastianreyes/.local/share/virtualenvs/machinelearnin
g_zoomcamp-axH7onbe, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, 
wheel=bundle, via=copy, app_data_dir=/Users/sebastianreyes/Library/Application 
Support/virtualenv)
    added seed packages: pip==24.2, setuptools==75.2.0, wheel==0.44.0
  activators 
BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator
,PythonActivator

✔ Successfully created virtual environment!
Virtualenv location: /Users/sebastianreyes/.local/share/virtualenvs/mac

requirements.txt found in /Users/sebastianreyes/Documents/3 
Carrera/Proyectos/Portafolio instead of Pipfile! Converting...
✔ Success!
did. 
We recommend updating your Pipfile to specify the "*" version, instead.
Building requirements...
Resolving dependencies...


Pipfile.lock not found, creating...
Locking [packages] dependencies...


✔ Success!
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.2...
✔ Installation Succeeded


Locking [dev-packages] dependencies...
Updated Pipfile.lock (857e269dd5e6f9241942764fe636bf27f2950848786ca9202c3046f03b77355c)!


Building requirements...
Resolving dependencies...
✔ Success!
Building requirements...
Resolving dependencies...
✔ Success!
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (77355c)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (77355c)...
First hash for scikit-learn: sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445


In [7]:
import requests
import os

def download_file(url, filename):
    try:
        # Send GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Save the content to a file
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded {filename}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {filename}: {str(e)}")

def main():
    # Define the prefix URL
    prefix = "https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework"
    
    # List of files to download
    files = {
        "model1.bin": f"{prefix}/model1.bin",
        "dv.bin": f"{prefix}/dv.bin"
    }
    
    # Download each file
    for filename, url in files.items():
        download_file(url, filename)

if __name__ == "__main__":
    main()

Successfully downloaded model1.bin
Successfully downloaded dv.bin


In [8]:
import pickle

def load_models(model_path='model1.bin', dv_path='dv.bin'):
    try:
        # Load the model
        with open(model_path, 'rb') as model_file:
            model = pickle.load(model_file)
        print("Model loaded successfully")
        
        # Load the DictVectorizer
        with open(dv_path, 'rb') as dv_file:
            dv = pickle.load(dv_file)
        print("DictVectorizer loaded successfully")
        
        return model, dv
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {str(e)}")
        return None, None
    except Exception as e:
        print(f"Error loading models: {str(e)}")
        return None, None

if __name__ == "__main__":
    # Load the models
    model, dv = load_models()
    
    # Print some basic information about the loaded objects
    if model is not None and dv is not None:
        print("\nModel type:", type(model))
        print("DictVectorizer type:", type(dv))

Model loaded successfully
DictVectorizer loaded successfully

Model type: <class 'sklearn.linear_model._logistic.LogisticRegression'>
DictVectorizer type: <class 'sklearn.feature_extraction._dict_vectorizer.DictVectorizer'>


# Question 3

In [9]:
import pickle

def load_models(model_path='model1.bin', dv_path='dv.bin'):
    with open(model_path, 'rb') as model_file:
        model = pickle.load(model_file)
    with open(dv_path, 'rb') as dv_file:
        dv = pickle.load(dv_file)
    return model, dv

def score_client(client_data):
    try:
        # Load the models
        model, dv = load_models()
        
        # Transform the client data using DictVectorizer
        X = dv.transform([client_data])
        
        # Make prediction
        prediction = model.predict_proba(X)[0, 1]
        
        # Format the probability as a percentage
        probability = prediction * 100
        
        return {
            'client_data': client_data,
            'probability': f"{probability:.2f}%"
        }
        
    except Exception as e:
        return f"Error making prediction: {str(e)}"

if __name__ == "__main__":
    # Client data
    client = {
        "job": "management",
        "duration": 400,
        "poutcome": "success"
    }
    
    # Get the prediction
    result = score_client(client)
    print("\nPrediction result:")
    print(f"Client data: {result['client_data']}")
    print(f"Probability of positive outcome: {result['probability']}")


Prediction result:
Client data: {'job': 'management', 'duration': 400, 'poutcome': 'success'}
Probability of positive outcome: 75.91%


# Question 4

In [11]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [19]:

pip install gunicorn

Note: you may need to restart the kernel to use updated packages.


In [23]:
from flask import Flask, request, jsonify
import pickle

# Initialize Flask app
app = Flask('credit_scoring')

# Load the saved model and DictVectorizer when the app starts
def load_models():
    with open('model1.bin', 'rb') as f_model:
        model = pickle.load(f_model)
    with open('dv.bin', 'rb') as f_dv:
        dv = pickle.load(f_dv)
    return model, dv

# Load models globally
model, dv = load_models()

@app.route('/predict', methods=['POST'])
def predict():
    # Get client data from the request
    client = request.get_json()
    
    # Prepare the data using the DictVectorizer
    X = dv.transform([client])
    
    # Make prediction using the model
    prediction = model.predict_proba(X)[0, 1]
    
    # Return the result as JSON
    result = {
        'probability': float(prediction),
        'probability_percentage': f"{prediction * 100:.2f}%"
    }
    
    return jsonify(result)

@app.route('/ping', methods=['GET'])
def ping():
    return jsonify({'status': 'ok'})

In [22]:
gunicorn --bind 0.0.0.0:9696 predict:app

SyntaxError: invalid syntax (2854942830.py, line 1)

# Question 5

In [24]:
pip install docker

Note: you may need to restart the kernel to use updated packages.


In [27]:
import subprocess
import sys

def install_and_test_docker():
    try:
        # Install docker SDK for Python
        print("Installing Docker SDK for Python...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "docker"])
        
        # Verify installation
        print("\nVerifying installation...")
        import docker
        
        # Try to connect to Docker daemon
        client = docker.from_env()
        
        # Get Docker version
        version = client.version()
        print("\nDocker installation successful!")
        print(f"Docker version: {version['Version']}")
        print(f"API version: {version['ApiVersion']}")
        
        # List containers
        print("\nActive containers:")
        for container in client.containers.list():
            print(f"- {container.name}")
        
    except subprocess.CalledProcessError as e:
        print(f"Error during installation: {str(e)}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    install_and_test_docker()

Installing Docker SDK for Python...

Verifying installation...
Error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))


In [33]:
import docker
import time

def check_docker_status():
    try:
        # Initialize Docker client
        print("Checking Docker connection...")
        client = docker.from_env()
        
        # Test connection
        client.ping()
        print("Docker is running!")
        
        # Now try to pull the image
        image_name = "svizor/zoomcamp-model:3.11.5-slim"
        print(f"\nPulling image {image_name}...")
        image = client.images.pull(image_name)
        
        # Get image size
        size_bytes = image.attrs['Size']
        size_mb = size_bytes / (1024 * 1024)  # Convert to MB
        
        print(f"Image pulled successfully!")
        print(f"Image size: {size_mb:.2f} MB")
        
    except docker.errors.DockerException as e:
        print("Error: Docker is not running. Please start Docker Desktop first.")
        print("If Docker Desktop is not installed, please install it from:")
        print("https://www.docker.com/products/docker-desktop/")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    check_docker_status()

Checking Docker connection...
Docker is running!

Pulling image svizor/zoomcamp-model:3.11.5-slim...
Image pulled successfully!
Image size: 124.45 MB
